<a href="https://colab.research.google.com/github/Deadly-Smile/Data-Science-Staff/blob/main/test_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%
import os
import cv2
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import accuracy_score

#%%
def smash_and_reconstruct(image, patch_size=32):
    """
    Splits the image into patches, separates rich and poor texture regions,
    and reconstructs two images: one for rich and one for poor textures.

    Args:
        image (numpy.ndarray): Input image of shape (H, W, 3) in RGB format.
        patch_size (int): Size of each patch (default: 32).

    Returns:
        rich_texture (numpy.ndarray): Reconstructed image with rich texture patches.
        poor_texture (numpy.ndarray): Reconstructed image with poor texture patches.
    """
    height, width, channels = image.shape
    assert channels == 3, "Input image must have 3 channels (RGB)."

    # Pad image to be divisible by patch_size
    pad_height = (patch_size - height % patch_size) % patch_size
    pad_width = (patch_size - width % patch_size) % patch_size
    image = np.pad(image, ((0, pad_height), (0, pad_width), (0, 0)), mode='reflect')

    # Split into patches
    patches = [
        image[i:i+patch_size, j:j+patch_size]
        for i in range(0, image.shape[0], patch_size)
        for j in range(0, image.shape[1], patch_size)
    ]

    # Placeholder for texture analysis
    rich_texture_patches = []
    poor_texture_patches = []

    for patch in patches:
        # Calculate a "texture richness" score (variance of Laplacian)
        gray_patch = cv2.cvtColor(patch, cv2.COLOR_RGB2GRAY)
        texture_score = cv2.Laplacian(gray_patch, cv2.CV_64F).var()

        # Arbitrary threshold: consider patches with high variance as "rich textures"
        if texture_score > 50:  # Adjust threshold based on experimentation
            rich_texture_patches.append(patch)
            poor_texture_patches.append(np.zeros_like(patch))  # Placeholder
        else:
            poor_texture_patches.append(patch)
            rich_texture_patches.append(np.zeros_like(patch))  # Placeholder

    # Reconstruct images
    rich_texture = reconstruct_image(rich_texture_patches, image.shape[:2], patch_size)
    poor_texture = reconstruct_image(poor_texture_patches, image.shape[:2], patch_size)

    return rich_texture, poor_texture

def reconstruct_image(patches, image_shape, patch_size):
    """
    Reconstructs an image from patches.

    Args:
        patches (list of numpy.ndarray): List of patches.
        image_shape (tuple): Shape of the original image (H, W).
        patch_size (int): Size of each patch.

    Returns:
        numpy.ndarray: Reconstructed image.
    """
    reconstructed = np.zeros((image_shape[0], image_shape[1], 3), dtype=np.uint8)
    idx = 0
    for i in range(0, image_shape[0], patch_size):
        for j in range(0, image_shape[1], patch_size):
            reconstructed[i:i+patch_size, j:j+patch_size] = patches[idx]
            idx += 1
    return reconstructed

#%%
# class TextureDataset(Dataset):
#     def __init__(self, root_dir, transform=None, patch_size=32, fraction=1.0):
#         self.root_dir = root_dir
#         self.transform = transform
#         self.patch_size = patch_size
#         self.data = []
#         self.labels = []
#
#         # Load dataset
#         for label, sub_dir in enumerate(['0_real', '1_fake']):
#             folder = os.path.join(root_dir, sub_dir)
#             files = os.listdir(folder)
#             # Use only a fraction of the dataset
#             sampled_files = files[:int(len(files) * fraction)]
#             for file in sampled_files:
#                 filepath = os.path.join(folder, file)
#                 self.data.append(filepath)
#                 self.labels.append(label)
#
#     def __len__(self):
#         return len(self.data)
#
#     def __getitem__(self, idx):
#         filepath = self.data[idx]
#         label = self.labels[idx]
#
#         # Load and process image
#         image = cv2.imread(filepath)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#
#         # Smash and Reconstruct
#         rich_texture, poor_texture = smash_and_reconstruct(image, self.patch_size)
#
#         if self.transform:
#             rich_texture = self.transform(rich_texture)
#             poor_texture = self.transform(poor_texture)
#
#         # Concatenate rich and poor textures along the batch dimension
#         # Each sample will now have shape [6, height, width]
#         combined_textures = torch.cat([rich_texture, poor_texture], dim=0)
#
#         return combined_textures, label

class TextureDataset(Dataset):
    def __init__(self, root_dir, transform=None, patch_size=32, fraction=1.0, save_dir=None, save_preprocessed=False):
        self.root_dir = root_dir
        self.transform = transform
        self.patch_size = patch_size
        self.data = []
        self.labels = []
        self.save_dir = save_dir
        self.save_preprocessed = save_preprocessed

        # Load dataset
        for label, sub_dir in enumerate(['0_real', '1_fake']):
            folder = os.path.join(root_dir, sub_dir)
            files = os.listdir(folder)
            sampled_files = files[:int(len(files) * fraction)]
            for file in sampled_files:
                filepath = os.path.join(folder, file)
                self.data.append(filepath)
                self.labels.append(label)

        if self.save_preprocessed and self.save_dir:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        filepath = self.data[idx]
        label = self.labels[idx]

        # Check if preprocessed data exists
        if self.save_dir:
            save_path = os.path.join(self.save_dir, f"{idx}.pt")
            if os.path.exists(save_path):
                data = torch.load(save_path)
                return data["textures"], data["label"]

        # Load and process image
        image = cv2.imread(filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Smash and Reconstruct
        rich_texture, poor_texture = smash_and_reconstruct(image, self.patch_size)

        if self.transform:
            rich_texture = self.transform(rich_texture)
            poor_texture = self.transform(poor_texture)

        # Concatenate textures
        combined_textures = torch.cat([rich_texture, poor_texture], dim=0)

        if self.save_preprocessed and self.save_dir:
            # Save preprocessed data
            torch.save({"textures": combined_textures, "label": label}, save_path)

        return combined_textures, label

#%%
from torchvision.models import ResNet50_Weights
import torch
import torch.nn as nn
import torchvision.models as models

class TextureResNet(nn.Module):
    def __init__(self):
        super(TextureResNet, self).__init__()
        # Load pretrained ResNet-50
        self.resnet = models.resnet50(weights=ResNet50_Weights.DEFAULT)

        # Update the first convolution layer to handle 6 input channels
        self.resnet.conv1 = nn.Conv2d(6, 64, kernel_size=7, stride=2, padding=3, bias=False)

        # Replace the final fully connected layer for binary classification
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, 2)

    def forward(self, x):
        return self.resnet(x)

#%%
def train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=5):
    model.to(device)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        for textures, labels in train_loader:
            textures, labels = textures.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(textures)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

        # Validation phase
        validate_model(model, val_loader, device)

def validate_model(model, val_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for textures, labels in val_loader:
            textures, labels = textures.to(device), labels.to(device)
            outputs = model(textures)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    print(f"Validation Accuracy: {acc:.4f}")

#%%
# Parameters
root_dir = "/content/drive/MyDrive/airplane"
batch_size = 16
num_epochs = 25
learning_rate = 0.0001  # Lower value for fine-tuning ResNet
fraction = 1  # Use 30% of the dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally
    transforms.RandomRotation(degrees=15),  # Randomly rotate the image by ±15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust color properties
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),
])

# Load dataset (using a fraction)
dataset = TextureDataset(root_dir, transform=transform, fraction=fraction, save_dir="airplane_processed")
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

#%%
# Model, Loss, Optimizer
model = TextureResNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  # Explicit learning rate

# Train and Validate
train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs)
# model.summary()
torch.save(model, "texture-based_1.pth")
torch.save(model.state_dict(), "texture-based_2.pth")


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 113MB/s]


In [ ]:
#%%
import os
from sklearn.metrics import accuracy_score

def evaluate_model_on_dataset(model_path, root_dir, device, patch_size=32):
    # Load model
    model = TextureResNet()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    model.to(device)

    # Transformations
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    all_preds = []
    all_labels = []

    # Iterate through subdirectories (0_real and 1_fake)
    for label, sub_dir in enumerate(['0_real', '1_fake']):
        sub_dir_path = os.path.join(root_dir, sub_dir)
        if not os.path.exists(sub_dir_path):
            print(f"Directory {sub_dir_path} does not exist. Skipping...")
            continue

        for image_name in os.listdir(sub_dir_path):
            image_path = os.path.join(sub_dir_path, image_name)
            if not os.path.isfile(image_path):
                continue  # Skip non-file entries

            # Load and process image
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Smash and Reconstruct
            rich_texture, poor_texture = smash_and_reconstruct(image, patch_size)
            rich_texture = transform(rich_texture)
            poor_texture = transform(poor_texture)

            # Concatenate textures
            combined_textures = torch.cat([rich_texture, poor_texture], dim=0).unsqueeze(0).to(device)

            # Prediction
            with torch.no_grad():
                output = model(combined_textures)
                _, prediction = torch.max(output, 1)

            all_preds.append(prediction.item())
            all_labels.append(label)

    # Calculate and print accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy on the dataset: {accuracy:.4f}")

# Example Usage
root_dir = "/content/drive/MyDrive/airplane"  # Replace with your test dataset root
# model_path = "texture-based_1.pth"
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# evaluate_model_on_dataset(model_path, root_dir, device)
model_path = "texture-based_1.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
evaluate_model_on_dataset(model_path, root_dir, device)